In [ ]:
def mod_display(filegrb,feature,style="gray",index=0,crop=True,scale=True):
    """
    filegrb: a string containing the filepath to the file that ends in .grb or .grb2
    feature: a string containing the feature to be displayed
    style: a string containing the cmap coloration
    index: an integer used to denote several versions of the feature variable
    crop: a boolean variable used to show the cropped image
    scale: a boolean variable used to scale the data down between 0 and 1
    """
    if (crop == True):
        import pygrib
        import matplotlib.pyplot as plt
        lat_bot = 31.0
        lat_top = 41.0
        lon_left = -102.0
        lon_right = -82.0
        grbs = pygrib.open(filegrb)
        grb = grbs.select(name=feature)[index]
        grbs.close()
        data,lats,lons = grb.data(lat1=lat_bot,lat2=lat_top,lon1=lon_left,lon2=lon_right)
        if scale==True:
            data = (data/data.max())*10
        plt.scatter(x=lons,y=lats,s=data,c=data,marker=",",cmap=style,alpha=1.0)
        plt.title(feature + " of Domain")
        plt.xlabel("Longitude W")
        plt.ylabel("Latitude N")
        plt.colorbar()
    else:
        import pygrib
        import matplotlib.pyplot as plt
        grbs = pygrib.open(filegrb)
        grb = grbs.select(name=feature)[index]
        grbs.close()
        data = grb.values
        if scale==True:
            data = data/data.max()
        plt.imshow(data,cmap=style,origin="lower")
        plt.title(feature + " of NA")
        plt.xlabel("Longitude W")
        plt.ylabel("Latitude N")
        plt.colorbar()

In [ ]:
filegrb = "/mnt/data-weather/moddata/modgrb/namanl_218_20171228_1200_000.grb2"
feature = "Temperature"

In [ ]:
mod_display(filegrb,feature)

In [ ]:
mod_display(filegrb,feature,crop=False)

In [ ]:
import pygrib
grbs = pygrib.open(filegrb)
for grb in grbs:
    print(grb)
grbs.close()

In [ ]:
mod_display(filegrb,"Geopotential Height",crop=False)

In [ ]:
mod_display(filegrb,"Geopotential Height")